# The Architect's Co-Pilot for Adaptive Retail Design with GCP and Gemini

In [ ]:

# !python -m pip install --quiet python-dotenv google-generativeai pinecone pypdf tiktoken langgraph pydantic pytrends mlflow matplotlib langchain-pinecone


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.10 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:

import os, json, time, math, hashlib, uuid
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv; load_dotenv()

import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec
from pypdf import PdfReader
import mlflow

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from pydantic import BaseModel
from langgraph.graph import StateGraph, END
from pytrends.request import TrendReq

required_env = ["GOOGLE_API_KEY", "GOOGLE_GENAI_USE_VERTEXAI", "GOOGLE_CLOUD_LOCATION", "GOOGLE_CLOUD_PROJECT", "PINECONE_API_KEY"]
missing = [k for k in required_env if not os.getenv(k)]
if missing: raise EnvironmentError(f"Missing env vars: {missing}")
assert os.getenv("GOOGLE_GENAI_USE_VERTEXAI") == "False"

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
EMBED_MODEL = "text-embedding-004"
GEN_MODEL   = "gemini-2.0-flash"

PINECONE_INDEX_NAME = "blue-retail-docs"
PINECONE_CLOUD = "aws"; PINECONE_REGION = "us-east-1"; VECTOR_DIM = 768

# DOCS_DIR = "./docs"; OUTPUTS_DIR = "./outputs"
# os.makedirs(DOCS_DIR, exist_ok=True); os.makedirs(OUTPUTS_DIR, exist_ok=True)
# print("Ready with:", EMBED_MODEL, GEN_MODEL, PINECONE_INDEX_NAME)


In [3]:

def sha256_text(s: str) -> str:
    import hashlib; return hashlib.sha256(s.encode("utf-8")).hexdigest()

def read_pdf_text(path: str) -> List[Dict[str, Any]]:
    pages = []
    with open(path, "rb") as f:
        pdf = PdfReader(f)
        for i, page in enumerate(pdf.pages, start=1):
            try: txt = page.extract_text() or ""
            except Exception: txt = ""
            pages.append({"page": i, "text": txt})
    return pages

def read_txt(path: str) -> str:
    with open(path, "r", encoding="utf-8", errors="ignore") as f: return f.read()

def recursive_chunk(text: str, max_chars: int = 2000, overlap: int = 200) -> List[str]:
    chunks=[]; i=0; n=len(text)
    while i<n:
        end=min(i+max_chars,n); chunk=text[i:end]; chunks.append(chunk.strip()); i=end-overlap
        if i<0: i=0
        if i>=n: break
    return [c for c in chunks if c]


In [4]:

def embed_texts(texts: List[str], model: str = EMBED_MODEL) -> List[List[float]]:
    out=[]
    for t in texts:
        resp = genai.embed_content(model=model, content=t)
        out.append(resp["embedding"])
    return out

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
existing = [idx["name"] for idx in pc.list_indexes()]
if PINECONE_INDEX_NAME not in existing:
    print("Creating Pinecone index...")
    pc.create_index(name=PINECONE_INDEX_NAME, dimension=VECTOR_DIM, metric="cosine",
                    spec=ServerlessSpec(cloud=PINECONE_CLOUD, region=PINECONE_REGION))
    while True:
        if pc.describe_index(PINECONE_INDEX_NAME).status["ready"]: break
        time.sleep(2)
index = pc.Index(PINECONE_INDEX_NAME)
print("Index ready:", PINECONE_INDEX_NAME)


Creating Pinecone index...
Index ready: blue-retail-docs


In [ ]:

DOC_PATHS = [
    # "./docs/Blue_Retail_Brand_Book_v4.pdf",
    # "./docs/Fixture_Catalog_Q3_2025.pdf",
    # "./docs/National_Building_Code_Accessibility_Chapter.txt",
    # "./docs/Store_Leasing_Agreement_Surat.pdf",
    # "./docs/Retail_Design_Best_Practices.md",
]

def detect_doctype(path: str) -> str:
    name=os.path.basename(path).lower()
    if "brand" in name: return "brand"
    if "fixture" in name: return "fixture"
    if "leasing" in name or "lease" in name: return "lease"
    if "accessibility" in name or "nbc" in name: return "nbc"
    if name.endswith(".md"): return "best_practices"
    if name.endswith(".txt"): return "text"
    if name.endswith(".pdf"): return "pdf"
    return "other"

def ingest_documents(paths: List[str]):
    items=[]
    for p in paths:
        if not os.path.exists(p):
            print("[WARN] Missing:", p); continue
        doctype=detect_doctype(p); base=os.path.basename(p)
        if p.lower().endswith(".pdf"):
            for page_obj in read_pdf_text(p):
                page=page_obj["page"]; text=page_obj["text"]
                if not text.strip(): continue
                for i,ch in enumerate(recursive_chunk(text,1800,200)):
                    cid=f"{base}-{page}-{i}-{uuid.uuid4().hex[:8]}"
                    items.append((cid,ch,{"source_document":base,"doctype":doctype,"page":page}))
        elif p.lower().endswith(".txt") or p.lower().endswith(".md"):
            content=read_txt(p); 
            for i,ch in enumerate(recursive_chunk(content,1800,200)):
                cid=f"{base}-{i}-{uuid.uuid4().hex[:8]}"
                items.append((cid,ch,{"source_document":base,"doctype":doctype}))
    if not items:
        print("No content collected. Add files to DOC_PATHS and re-run."); return
    texts=[t for _,t,_ in items]; embeds=embed_texts(texts)
    vecs=[]
    for (cid,text,meta),vec in zip(items,embeds):
        m=dict(meta); m.update({"hash":sha256_text(text),"len":len(text)})
        vecs.append({"id":cid,"values":vec,"metadata":m})
    for i in range(0,len(vecs),100):
        index.upsert(vectors=vecs[i:i+100])
    print("Upserted", len(vecs), "chunks.")

ingest_documents(DOC_PATHS)


In [ ]:

def pinecone_search(query: str, top_k: int = 8, filter_meta: Optional[Dict[str, Any]] = None):
    qv = embed_texts([query])[0]
    return index.query(vector=qv, top_k=top_k, include_metadata=True, filter=filter_meta or {})
print("Smoke query:", [m["id"] for m in pinecone_search("emergency exit",3).get("matches",[])])


In [ ]:

def get_city_trend_weights(city: str, categories: List[str]) -> Dict[str, float]:
    pytrends = TrendReq(hl="en-US", tz=330)
    try:
        pytrends.build_payload(kw_list=categories, timeframe='now 7-d', geo='IN')
        df = pytrends.interest_over_time()
        if df is None or df.empty: raise ValueError("Empty trends")
        latest = df.iloc[-1][categories].to_dict()
    except Exception:
        latest = {k: 50 for k in categories}
    mx = max(latest.values()) if latest else 1.0
    if mx == 0: mx = 1.0
    return {k: round(v/mx,2) for k,v in latest.items()}

DEFAULT_CATEGORIES = ["mobiles","laptops","audio","gaming","appliances"]
get_city_trend_weights("Surat", DEFAULT_CATEGORIES)


In [ ]:

# Pydantic schema
class EntryPoint(BaseModel):
    id: str; x: float; y: float; width_m: float
class ZoneRect(BaseModel):
    x: float; y: float; w: float; h: float
class Zone(BaseModel):
    zone_id: str; name: str; priority: float; area_sqm: float; rect: ZoneRect
class Fixture(BaseModel):
    fixture_id: str; zone_id: str; x: float; y: float; w: float; h: float; rotation_deg: float = 0
class ComplianceReport(BaseModel):
    brand_rules_ok: bool; lease_ok: bool; nbc_ok: bool; violations: List[Dict[str, Any]]
class LayoutPlan(BaseModel):
    site: Dict[str, Any]
    zoning: List[Zone]
    fixtures: List[Fixture]
    paths: Dict[str, Any]
    constraints: Dict[str, Any]
    citations: List[Dict[str, Any]]
    compliance_report: Optional[ComplianceReport] = None


In [ ]:

def build_context_from_matches(matches: List[Dict[str, Any]], max_lines: int = 20) -> str:
    lines=[]
    for m in matches[:max_lines]:
        md=m.get("metadata",{}); src=md.get("source_document","unknown"); page=md.get("page","?")
        lines.append(f"[{m['id']}] {src} p.{page}")
    return "\n".join(lines)

def strategist_generate_layout(city: str, floor_area_sqm: float, entry_points: List[Dict[str, Any]], trend_weights: Dict[str,float],
                               top_k:int=12, corridor_min_m: float=1.2) -> LayoutPlan:
    matches = pinecone_search(f"layout constraints for {city}; NBC; lease; brand rules; fixtures", top_k=top_k).get("matches",[])
    citations=[{"chunk_id":m.get("id"),"source_document":m.get("metadata",{}).get("source_document"),"page":m.get("metadata",{}).get("page")} for m in matches]
    retrieval_context = build_context_from_matches(matches)
    user_prompt = (
        "You are the Layout Strategist for a premium electronics retailer. "
        "Synthesize a compliant conceptual layout only from retrieved documents and trend weights. "
        "Return STRICT JSON matching the schema (zones, fixtures, paths with min corridor width, constraints, citations).\n\n"
        f"City: {city}\nFloor area (sqm): {floor_area_sqm}\nEntry points: {entry_points}\n"
        f"Trend weights (0..1): {trend_weights}\nMin corridor width: {corridor_min_m} m\n"
        "Include 'citations' that reference the chunk ids."
    )
    model = genai.GenerativeModel(GEN_MODEL)
    last_err=None
    for _ in range(3):
        try:
            g = model.generate_content([retrieval_context, user_prompt])
            data = json.loads(g.text)
            plan = LayoutPlan(**data)
            if not plan.citations: plan.citations = citations
            return plan
        except Exception as e:
            last_err=e; time.sleep(1)
    raise RuntimeError(f"Failed to produce valid layout JSON: {last_err}")


In [ ]:

def compliance_check(plan: LayoutPlan) -> ComplianceReport:
    min_w = plan.paths.get("min_corridor_width_m", 1.2)
    violations=[]
    for c in plan.paths.get("corridors", []):
        width=min(c["w"], c["h"])
        if width < min_w:
            violations.append({"type":"corridor_width","required_m":float(min_w),"actual_m":float(width),"location":c})
    ok = len(violations)==0
    return ComplianceReport(brand_rules_ok=True, lease_ok=True, nbc_ok=ok, violations=violations)

def render_layout_png(plan: LayoutPlan, out_path: str):
    site=plan.site; dims=site.get("dimensions_m") or {"width": math.sqrt(site.get("floor_area_sqm", 200)), "height": math.sqrt(site.get("floor_area_sqm", 200))/1.5}
    W=dims["width"]; H=dims["height"]
    fig,ax=plt.subplots(figsize=(10,6)); ax.add_patch(Rectangle((0,0),W,H,fill=False,linewidth=2))
    for e in site.get("entry_points", []):
        ax.add_patch(Rectangle((e["x"], e["y"]-0.1),0.2,0.2,fill=True)); ax.text(e["x"]+0.25,e["y"],f"Entry {e['id']}",va='center')
    for z in plan.zoning:
        r=z.rect; ax.add_patch(Rectangle((r.x,r.y),r.w,r.h,fill=False)); ax.text(r.x+r.w/2,r.y+r.h/2,z.name,ha='center',va='center',fontsize=9)
    for fx in plan.fixtures:
        ax.add_patch(Rectangle((fx.x,fx.y),fx.w,fx.h,fill=True)); ax.text(fx.x+fx.w/2,fx.y+fx.h/2,fx.fixture_id,ha='center',va='center',fontsize=6)
    for c in plan.paths.get("corridors", []):
        ax.add_patch(Rectangle((c["x"],c["y"]),c["w"],c["h"],fill=False,linestyle='--')); ax.text(c["x"]+c["w"]/2,c["y"]+c["h"]/2,'Corridor',ha='center',va='center',fontsize=7)
    ax.set_xlim(0,W); ax.set_ylim(0,H); ax.set_aspect('equal', adjustable='box'); ax.set_title(f"Conceptual Layout – {site.get('city','')}")
    ax.set_xlabel("Meters (X)"); ax.set_ylabel("Meters (Y)"); plt.tight_layout(); plt.savefig(out_path,dpi=150); plt.close(fig)


In [ ]:

def log_run_to_mlflow(city: str, trend_weights: Dict[str,float], plan: LayoutPlan, png_path: str, index_name: str):
    mlflow.set_experiment("architect_copilot")
    with mlflow.start_run(run_name=f"layout_{city}"):
        mlflow.log_param("city", city); mlflow.log_param("index", index_name)
        mlflow.log_param("embed_model", EMBED_MODEL); mlflow.log_param("gen_model", GEN_MODEL)
        mlflow.log_param("corridor_min_m", plan.paths.get("min_corridor_width_m", 1.2))
        for k,v in trend_weights.items(): mlflow.log_param(f"trend_{k}", v)
        comp = plan.compliance_report.model_dump() if plan.compliance_report else {}
        mlflow.log_metric("nbc_ok", int(comp.get("nbc_ok", 0))); mlflow.log_metric("violations_count", len(comp.get("violations", [])) if comp else 0)
        json_path=os.path.join("./outputs", f"layout_plan_{city}.json")
        with open(json_path,"w") as f: json.dump(plan.model_dump(), f, indent=2)
        mlflow.log_artifact(json_path); 
        if os.path.exists(png_path): mlflow.log_artifact(png_path)
        rid = mlflow.active_run().info.run_id
    return rid


In [ ]:

CITY="Surat"; FLOOR_AREA_SQM=240.0; ENTRY_POINTS=[{"id":"E1","x":0.0,"y":6.0,"width_m":2.4}]; CATEGORIES=DEFAULT_CATEGORIES; CORRIDOR_MIN=1.2
trend_weights = get_city_trend_weights(CITY, CATEGORIES); print("Trend weights:", trend_weights)
try:
    plan = strategist_generate_layout(CITY, FLOOR_AREA_SQM, ENTRY_POINTS, trend_weights, top_k=10, corridor_min_m=CORRIDOR_MIN)
except RuntimeError as e:
    print("[WARN]", e, "— using baseline sample.")
    plan = LayoutPlan(
        site={"city": CITY, "floor_area_sqm": FLOOR_AREA_SQM, "dimensions_m":{"width":20.0,"height":12.0}, "entry_points": ENTRY_POINTS},
        zoning=[
            {"zone_id":"Z1","name":"Mobiles","priority":0.92,"area_sqm":60.0,"rect":{"x":1.0,"y":6.5,"w":9.0,"h":4.5}},
            {"zone_id":"Z2","name":"Laptops","priority":0.81,"area_sqm":50.0,"rect":{"x":1.0,"y":1.0,"w":9.0,"h":4.5}},
            {"zone_id":"Z3","name":"Audio","priority":0.68,"area_sqm":35.0,"rect":{"x":11.0,"y":1.0,"w":8.0,"h":5.0}},
            {"zone_id":"Z4","name":"Gaming","priority":0.55,"area_sqm":30.0,"rect":{"x":11.0,"y":7.0,"w":8.0,"h":4.5}}
        ],
        fixtures=[
            {"fixture_id":"FX-Table-120x80","zone_id":"Z1","x":2.0,"y":7.0,"w":1.2,"h":0.8},
            {"fixture_id":"FX-Table-120x80","zone_id":"Z1","x":4.0,"y":7.0,"w":1.2,"h":0.8},
            {"fixture_id":"FX-Table-120x80","zone_id":"Z2","x":2.0,"y":2.0,"w":1.2,"h":0.8},
            {"fixture_id":"FX-Table-120x80","zone_id":"Z2","x":4.0,"y":2.0,"w":1.2,"h":0.8},
            {"fixture_id":"FX-Wall-Display","zone_id":"Z3","x":11.0,"y":1.0,"w":0.5,"h":5.0},
            {"fixture_id":"FX-Demo-Console","zone_id":"Z4","x":12.0,"y":8.0,"w":2.0,"h":1.0}
        ],
        paths={"min_corridor_width_m": CORRIDOR_MIN, "corridors":[{"x":10.2,"y":1.0,"w":0.8,"h":10.0},{"x":1.0,"y":5.7,"w":18.0,"h":0.8}]},
        constraints={"brand_rules": ["logo_sightline_from_entry"], "lease":["emergency_exit_clear_1.5m"], "nbc_accessibility":["ramp_slope_1_in_12"]},
        citations=[]
    )
plan.compliance_report = compliance_check(plan)
png_path=os.path.join("./outputs", f"layout_{CITY}.png"); render_layout_png(plan, png_path)
json_path=os.path.join("./outputs", f"layout_plan_{CITY}.json"); 
with open(json_path,"w") as f: json.dump(plan.model_dump(), f, indent=2)
print("Saved:", json_path, "|", png_path); print("Compliance:", plan.compliance_report.model_dump() if plan.compliance_report else None)


## (Deployment – commented for later)
Uncomment and adapt when deploying Streamlit/FastAPI to Cloud Run.

In [ ]:

# %%bash
# cat > Dockerfile <<'EOF'
# FROM python:3.11-slim
# WORKDIR /app
# COPY requirements.txt .
# RUN pip install -r requirements.txt
# COPY . .
# ENV PORT=8080
# CMD ["streamlit", "run", "ui/app.py", "--server.port=8080", "--server.address=0.0.0.0"]
# EOF
# echo "Dockerfile scaffold written."


In [ ]:

# # gcloud builds submit --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/architect-copilot:latest
# # gcloud run deploy architect-copilot --image gcr.io/${GOOGLE_CLOUD_PROJECT}/architect-copilot:latest --platform=managed --region=us-central1 # #   --set-env-vars GOOGLE_CLOUD_PROJECT=${GOOGLE_CLOUD_PROJECT},GOOGLE_CLOUD_LOCATION=global,GOOGLE_GENAI_USE_VERTEXAI=False # #   --set-secrets GOOGLE_API_KEY=GOOGLE_API_KEY:latest,PINECONE_API_KEY=PINECONE_API_KEY:latest
